# Extracting multimodal data features

`siibra` provides access to data features of different modalities using the `get_features` method, which accepts a feature modality and is sensitive to the selections configured in the atlas (parcellation, region). If not particular selection is made, `get_features` considers all brain regions of the current parcellation.

In [ ]:
import siibra
from os import environ

### 1. Setup `siibra` to connect to the EBRAINS Knowledge Graph access token

`siibra` retrieves some data from the EBRAINS Knowledge Graph, which requires
authentication. To do so, please follow these steps:

 1. If you do not yet have an EBRAINS account, register [here](https://ebrains.eu/register). As you are reading this notebook in the EBRAINS collaboratory, this is most probably not necessary at this point.
 2. Your EBRAINS account needs to be enabled for programmatic access to the EBRAINS Knowledge Graph to fetch metadata. This is formal step to acknowledge additional terms of use, and done quickly by emailing to the KG team. A link and template email to do so can be found right on top of the [Knowledge Graph developer page](https://kg.humanbrainproject.eu/develop.html).
 3. Create an authentication token for EBRAINS by visiting
[the EBRAINS authorization endpoint](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize). 
 4. Copy the token, and store it in the enviroment variable `HBP_AUTH_TOKEN` (just modify and execute the cell below accordingly).

Note that as of now, you have to get a new token (steps 3. and 4.) approximately every day.

In [ ]:
import webbrowser
if "HBP_AUTH_TOKEN" not in environ:
    webbrowser.open('https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize')
    token = input("Enter your token here, then press 'Enter': ")
    environ['HBP_AUTH_TOKEN'] = token

### 2. Extract transmitter receptor densities

Transmitter receptor density fingerprints are linked to brain regions by their name in the EBRAINS Knowledge Graph. Like any data feature, they are accessed using the `get_features` method of the atlas, which makes use of the current selection in the atlas. The `get_features` method knows from the specified data modality that the match is determined from the brain region identified. Receptor densities come as a nicely structured datatype. Amongst other things, they can visualize themselves in a plot.

If we don't specify a particular selection, the atlas will return all available receptor density features linked to the parcellation map:

In [ ]:
atlas = siibra.atlases['human']
atlas.select_parcellation('julich')
atlas.clear_selection()
features = atlas.get_features(
    siibra.modalities.ReceptorDistribution)
print("Receptor density features found for the following regions:")
print(", ".join({f.regionspec for f in features}))

If we select a region, the returned list is filtered accordingly. 

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.ReceptorDistribution)
for r in features:
    fig = r.plot(r.regionspec)

### 3. Extracting Gene Expressions from the Allen Atlas 

The atlas client can make calls to gene expression data from the Allen atlas and evaluate them in the ICBM space to find regional gene expression levels. It also has a list of available gene names for convenient selection. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. The `get_features` method detects this from the feature modality, and applies the mask of the regions that are currently selected in the atlas to filter the probes. We can visualize these filtered locations.

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.GeneExpression, 
    gene=siibra.features.gene_names.GABARAPL2)
print(features[0])

# plot
from nilearn import plotting
all_coords = [tuple(g.location) for g in features]
mask = atlas.build_mask(siibra.spaces.MNI152_2009C_NONL_ASYM)
display = plotting.plot_roi(mask)
display.add_markers(all_coords,marker_size=5) 

### 4. Retrieving connectivity matrices
For `siibra`, a connectivity matrix is a data feature like the others below, and can be found using the same `get_features` function just by choosing another modality. `siibra` knows from the modality type that this type of data does not match to the selected brain region in the atlas, but to the selected parcellation. Therefore, `get_features` return connectivity datasets that are defined for the selected parcellation. 

To learn about the nature of the provided connectivity, the `src_info` attribute provides a detailed description of the dataset. 

In [ ]:
# Get the first four connectivity matrices available for the parcellation
features = atlas.get_features(siibra.modalities.ConnectivityMatrix)[:4]

# format dataset names for use as figure titles
from textwrap import wrap
titleformat = lambda text : "\n".join(wrap(text.replace('_',' '),20)) 

# plot the matrices
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(12,20))
for i,feature in enumerate(features):
    ax = fig.add_subplot(1,len(features),i+1)
    ax.imshow(feature.array,cmap=plt.cm.viridis)
    ax.set_title(titleformat(feature.src_name),size=10)
fig.show()

### 5. Retrieving iEEG electrodes and contact points

#### Query by electrode

We start by querying for electrodes. Electrodes match a selected region if any of their contact points does. To see which contact points match specifically, we need to iterate electrodes.

In [ ]:
electrodes = atlas.get_features(siibra.modalities.IEEG_Electrode)

# we grep the MNI coordinates of contact points for visualization
contactpoints = [c for e in electrodes for c in e] 
coord = lambda contactpoint: contactpoint.location
color = lambda contactpoint:"red" if contactpoint.matches(atlas) else "gray"
plotting.view_markers(
    list(map(coord,contactpoints)), # list of coordinates
    list(map(color,contactpoints)), # list of colors
    marker_size=3)

##### Query by contact points

We can also directly query for contact points if we want to be more specific. Note that each contact point has a reference to its corresponding electrode via the `.electrode` attribute, so we could get access to neighboring contact points from here as well.

In [ ]:
# get all electrode contact points inside the selected region
contactpoints = atlas.get_features(siibra.modalities.IEEG_ContactPoint)
plotting.view_markers(list(map(coord,contactpoints)),marker_size=3)

### 6. Retrieving other types of regional datasets from EBRAINS

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(siibra.modalities.EbrainsRegionalDataset)
for f in features:
    print(f"- {f.name}")